In [60]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
seccion_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Secciones\Hogares_RegimenDeTenencia_Secciones_Censo_2011.csv"
seccion_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Secciones\Hogares_RegimenDeTenencia_Secciones_Censo_2021.csv"
atributo = "Regimen de Tenencia"
valores = "Hogares"
valoresTotal = "Total_hogares_seccion"
valoresPorcentaje = "Hogares %"
cartografia_seccion_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaSeccionCensal-25830\da_cartografiaSeccionCensalPolygon.shp"
export_seccion_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Secciones\2011_Secciones_RegimenDeTenencia.csv"
export_seccion_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Secciones\2021_Secciones_RegimenDeTenencia.csv"

## Lectura inicial de datos regimen de tenencia
df_seccion_2011 =  pd.read_csv(seccion_2011_csv_path,sep=";")
df_seccion_2021 =  pd.read_csv(seccion_2021_csv_path,sep=";")
df_cartografia_distrito = gpd.read_file(cartografia_seccion_shp_path)

# Ver las columnas
#print(df_seccion_2011.columns)
#print(df_seccion_2021.columns)

# Inspeccionar los primeros registros
#print(df_seccion_2011.head())
#print(df_seccion_2021.head())

#Cambio el nombre de la columna 'Sección ' a Seccion de residencia
df_seccion_2021.rename(columns={'Distrito ': 'Distrito de residencia'}, inplace=True)
df_seccion_2021.rename(columns={'Sección ': 'Seccion de residencia'}, inplace=True)

#Se realiza una normalizacion de los csv  
df_seccion_2011_long = pd.melt(df_seccion_2011, id_vars=["Municipio de residencia","Distrito de residencia","Seccion de residencia"], var_name=atributo, value_name=valores)
#print(df_seccion_2011_long.columns)
#print(df_seccion_2011_long.head())
df_seccion_2021_long = pd.melt(df_seccion_2021, id_vars=["Municipio de residencia","Distrito de residencia","Seccion de residencia"], var_name=atributo, value_name=valores)
#print(df_seccion_2021_long.columns)
#print(df_seccion_2021_long.head())

## Eliminar filas ##
#Eliminar filas de municipio Total
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long['Municipio de residencia'] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long['Municipio de residencia'] != 'Total']
#Eliminar filas de Seccion de residencia Total
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long['Distrito de residencia'] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long['Distrito de residencia'] != 'Total']
#Eliminar filas de Seccion de residencia Total
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long['Seccion de residencia'] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long['Seccion de residencia'] != 'Total']
#Eliminar total de atributo (Regimen de Tenencia)
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long[atributo] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long[atributo] != 'Total']

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_seccion_2011_long[valores].isna()).sum()
num_filas_con_asterisco_2011 = (df_seccion_2011_long[valores] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_seccion_2011_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#0
#print(num_filas_contiene_asterisco_2011)#100
num_filas_en_blanco_2021 = (df_seccion_2021_long[valores].isna()).sum()
num_filas_con_asterisco_2021 = (df_seccion_2021_long[valores] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_seccion_2021_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

## Añadir columnas ##
#Añadir columna NUMSECCENS con el NUM de la seccion
df_seccion_2011_long["NUMSECCENS"] = df_seccion_2011_long["Seccion de residencia"].str[-5:].astype(float)
df_seccion_2021_long["NUMSECCENS"] = df_seccion_2021_long["Seccion de residencia"].str[-5:].astype(float)

#print(df_seccion_2011_long.head(10))
#print(df_seccion_2021_long.head(10))

## Gestion de Nulos y valores extraños ##
# Limpiar los datos: eliminar el asterisco al final, pero dejar intacto el '*'
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].replace(r'(\d+)\*$', r'\1', regex=True)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].replace(r'(\d+)\*$', r'\1', regex=True)
#Sustituimos los valores * por NA
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].replace('*', np.nan)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].replace('*', np.nan)
#Columna Personas como float
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].astype(float)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].astype(float)

# Diccionario de mapeo 2011 a 2021
mapping_dict_2011_2021 = {
    'Total': 'Total',
    'Al menos un miembro es arrendatario de toda o parte de la vivienda': 'En alquiler',
    'Al menos un miembro es propietario de la vivienda': 'En propiedad',
    'Otro régimen de tenencia': 'Otro régimen de tenencia'
}

df_seccion_2011_long[atributo] = df_seccion_2011_long[atributo].map(mapping_dict_2011_2021)

##Calcular el total de personas por distrito y el porcentaje 
df_seccion_2011_long[valoresTotal] = df_seccion_2011_long.groupby('Seccion de residencia')[valores].transform('sum')
df_seccion_2021_long[valoresTotal] = df_seccion_2021_long.groupby('Seccion de residencia')[valores].transform('sum')
#Calcular porcentaje sobre el total
df_seccion_2011_long[valoresPorcentaje] = (df_seccion_2011_long[valores] / df_seccion_2011_long[valoresTotal])
df_seccion_2021_long[valoresPorcentaje] = (df_seccion_2021_long[valores] / df_seccion_2021_long[valoresTotal])
#print(df_seccion_2011_long[df_seccion_2011_long['NUMSECCENS'] == 5002])
#print(df_seccion_2011_long[df_seccion_2011_long['NUMSECCENS'] == 2017])
#print(df_seccion_2011_long[df_seccion_2011_long['NUMSECCENS'] == 2026])
#print(df_seccion_2021_long[df_seccion_2021_long['NUMSECCENS'] == 5002])

#Pivotar la tabla de nuevo para crear variables, una por categoria con el % ; una fila por distrito
df_seccion_2011_long  = df_seccion_2011_long.pivot(index='NUMSECCENS', columns=atributo, values=valoresPorcentaje)
df_seccion_2021_long  = df_seccion_2021_long.pivot(index='NUMSECCENS', columns=atributo, values=valoresPorcentaje)

print(df_seccion_2011_long)
print(df_seccion_2011_long.count())#43
print(df_seccion_2021_long)
print(df_seccion_2021_long.count())#185

Regimen de Tenencia  En alquiler  En propiedad  Otro régimen de tenencia
NUMSECCENS                                                              
2017.0                  0.027027      0.939189                  0.033784
2026.0                  0.109195      0.856322                  0.034483
2050.0                  0.114458      0.819277                  0.066265
5002.0                  0.495192      0.451923                  0.052885
6001.0                  0.150538      0.784946                  0.064516
6024.0                  0.081340      0.818182                  0.100478
7023.0                  0.043956      0.895604                  0.060440
7030.0                  0.058824      0.931373                  0.009804
7034.0                  0.270588      0.664706                  0.064706
7048.0                  0.124294      0.813559                  0.062147
7082.0                  0.000000      0.888268                  0.111732
7090.0                  0.071429      0.738095     

In [62]:
##Export 3 variables
df_seccion_2011_long.to_csv(export_seccion_2011_csv_path,index=True,sep=";")
df_seccion_2021_long.to_csv(export_seccion_2021_csv_path,index=True,sep=";")